In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler # standardize
from sklearn.model_selection import train_test_split
SEED = 100
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Cleaning

In [2]:
df = pd.read_csv('/kaggle/input/stat-441-w2022-data-challenge/train.csv')
df.head(10)

In [3]:
X = df.loc[:, ~df.columns.isin(['uniqueid', 'health'])]
y = df['health'].astype(str)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=SEED, stratify=y)

In [5]:
# drop columns 

col_too_many_null = []

for i in list(X_train.columns):
    if X_train[i].isna().sum()/len(X_train) >= 0.85:
        col_too_many_null.append(i)

len(col_too_many_null)
X_train = X_train.drop(col_too_many_null, axis=1)
X_test =  X_test.drop(col_too_many_null, axis=1)

In [ ]:
# draft
# pd.set_option('display.max_rows', None)
# X_train['xi7'] = X_train['x7'].fillna(X_train.groupby(['personid'])['x7'].transform('mean'))
# X_train[['personid', 'x7', 'xi7']].sort_values('personid')

# X_train['xi7'] = X_train.groupby('personid')['x7'].transform(lambda x: x.ffill().bfill())
# X_train[['personid', 'year', 'x7', 'xi7']].sort_values('personid')

In [6]:
# X_train.groupby(['personid', pd.Grouper(key='year')])['x7'].mean().ffill().reset_index()['x7']
X_train.sort_values('year')
X_test.sort_values('year')
for i in X_train.columns:
    X_train[i] = X_train.groupby('personid')[i].transform(lambda x: x.ffill().bfill())
    
for i in X_test.columns:
    X_test[i] = X_test.groupby('personid')[i].transform(lambda x: x.ffill().bfill())    

In [12]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)

In [5]:
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')

In [15]:
import sys
!{sys.executable} -m pip install missingpy

In [17]:
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [2]:
from missingpy import MissForest

# Make an instance and perform the imputation
imputer = MissForest(random_state=SEED, max_iter = 3)
imputer.fit(X_train)
imputed_train = imputer.transform(X_train)
imputed_test = imputer.transform(X_test)

In [ ]:
X_train = pd.DataFrame(imputed_train, columns=X_train.columns)
X_test = pd.DataFrame(imputed_test, columns = X_test.columns)

In [ ]:
X_train.isna().sum().sum()

In [ ]:
X_test.isna().sum().sum()

In [ ]:
print(len(X_train.columns))

# Convert Year column to categorical variable
X_train['year'] = X_train['year'].astype(str)
X_train = pd.get_dummies(X_train, columns=['year'], drop_first = True)
X_test['year'] = X_test['year'].astype(str)
X_test = pd.get_dummies(X_test, columns=['year'], drop_first = True)

print(len(X_train.columns))

In [ ]:
# for i in list(X_train.columns[1:]):
  #  X_train[i] = X_train[i].fillna(X_train.groupby(['personid'])[i].transform('mean'))

In [ ]:
X_train.head(5)

In [ ]:
X_test.iloc[:, 0:-6]

In [ ]:
X_test

In [ ]:
len(col_too_many_null)
len(X_train.columns)

In [ ]:
# PCA 

# Modelling

In [ ]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(X_train, y_train)
metrics.accuracy_score(y_test, model.predict(X_test))

In [ ]:
pd.value_counts(y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
clf = RandomForestClassifier(class_weight='balanced')
clf.fit(X_train, y_train)
params = {'n_estimators' : [100, 200],
          'max_depth' : [5, 8, 15, 25, 30, None],
          'min_samples_split' : [2, 5, 10, 15],
          'class_weight': ['balanced', None ]
}

random_search = RandomizedSearchCV(clf, params, n_iter=5, n_jobs=-1, scoring='accuracy', random_state=100 )
random_search.fit(X_train, y_train)
clf = random_search.best_estimator_

clf.score(X_test, y_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
svc_clf = SVC(gamma='auto', decision_function_shape='ovo')
svc_clf.fit(X_train, y_train)
svc_clf.score(X_test, y_test)

In [ ]:
lr = LogisticRegression(multi_class='auto', max_iter = 500)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

In [ ]:
import lightgbm as lgb
lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train)
lgbm.score(X_test, y_test)